In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from multilingual_pdf2text.pdf2text import PDF2Text
from multilingual_pdf2text.models.document_model.document import Document

pdf_document = Document(
        document_path='/home/rudra/Desktop/Multimodal rag/content_10/HSC26-Bangla1st-Paper.pdf',
        language='ben'
        )
pdf2text = PDF2Text(document=pdf_document)
content = pdf2text.extract()
print(content)

INFO:multilingual_pdf2text.doc2img.parse_document:Parsing document from pdf to image
INFO:multilingual_pdf2text.ocr.image_to_text:Extracting text from images via OCR


[{'page_number': 1, 'text': 'অপরিচিতা\n\nঅনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,\n\nকলকরো ৬ 86919\n'}, {'page_number': 2, 'text': "[115026]\nনী '৯1৬]খ7চ\n\nঅনলাইন ব্যাচ\n10401০:\nভ্ঁ শিখনফল\n\n৮ নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।\n\n৮ তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।\n\n৮ তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে।\n\n৮ তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্ট্য সম্পর্কে জ্ঞানলাভ করবে।\n৮ নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে\n\nপারবে।\n৮ মানুষ আশা নিয়ে বেঁচে থাকে- অনুপমের দৃষ্টান্তে মানবজীবনের এই চিরন্তন সত্যদর্শন সম্পর্কে\n\nজ্ঞানলাভ করবে।\n\nন্ট প্রাক-মূল্যায়ন\n\n১। অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?\n\nক) ডাক্তারি খ) ওকালতি গ) মাস্টারি ঘ) ব্যবসা\n\n২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার-\n\nক) প্রতিপত্তি খ) প্রভাব গ) বিচক্ষণতা ঘ) কুট বুদ্ধি\n\nনিচের অনুচ্ছেদটি পড়ে ৩ ও ৪ সংখ্যক প্রশ্নের উত্তর দাও।\n\nপিতৃহীন দীপুর চা

In [ ]:
import os
import re
from multilingual_pdf2text.pdf2text import PDF2Text
from multilingual_pdf2text.models.document_model.document import Document

# Function to detect paragraph boundaries
def is_paragraph_break(element, prev_element=None):
    """
    Identify paragraph breaks based on newlines, formatting, or metadata.
    Adjust based on multilingual_pdf2text output.
    """
    text = element.get('text', '').strip()
    metadata = element.get('metadata', {})
    
    # Check for empty lines, significant spacing, or metadata indicating a break
    is_empty = not text
    has_newline = metadata.get('newline', False) if metadata else False
    is_new_paragraph = text.startswith((' ', '\t')) or (prev_element and len(prev_element.get('text', '')) > 0 and text)
    
    return is_empty or has_newline or is_new_paragraph

# Function to chunk text by paragraphs with overlap
def chunk_by_paragraph(extracted_content, max_characters=1000, min_characters=200, overlap_characters=100):
    """
    Split text into paragraph-based chunks with overlap.
    """
    chunks = []
    current_chunk = []
    current_chunk_length = 0
    prev_element = None

    for element in extracted_content:
        text = element.get('text', '').strip()
        if not text:
            continue

        # Check for paragraph break
        if is_paragraph_break(element, prev_element) and current_chunk:
            chunk_text = " ".join(current_chunk)
            if len(chunk_text) >= min_characters:
                # Add chunk with overlap from previous chunk
                chunks.append({
                    "text": chunk_text,
                    "chunk_id": f"chunk_{len(chunks) + 1}",
                    "source": "story_pdf"
                })
                # Start new chunk with overlap
                overlap_text = chunk_text[-overlap_characters:] if len(chunk_text) > overlap_characters else chunk_text
                current_chunk = [overlap_text, text]
                current_chunk_length = len(overlap_text) + len(text)
            else:
                # Append to current chunk if too small
                current_chunk.append(text)
                current_chunk_length += len(text)
        else:
            current_chunk.append(text)
            current_chunk_length += len(text)

        # Split if chunk exceeds max_characters
        if current_chunk_length >= max_characters:
            chunk_text = " ".join(current_chunk)
            chunks.append({
                "text": chunk_text,
                "chunk_id": f"chunk_{len(chunks) + 1}",
                "source": "story_pdf"
            })
            # Start new chunk with overlap
            overlap_text = chunk_text[-overlap_characters:] if len(chunk_text) > overlap_characters else chunk_text
            current_chunk = [overlap_text]
            current_chunk_length = len(overlap_text)

        prev_element = element

    # Append the last chunk if it meets the minimum size
    if current_chunk and len(" ".join(current_chunk)) >= min_characters:
        chunks.append({
            "text": " ".join(current_chunk),
            "chunk_id": f"chunk_{len(chunks) + 1}",
            "source": "story_pdf"
        })

    return chunks

# Function to save chunks as text files
def save_chunks_to_files(chunks, output_dir):
    """
    Save each chunk as a text file named by chunk ID.
    """
    os.makedirs(output_dir, exist_ok=True)
    for chunk in chunks:
        file_name = f"{chunk['chunk_id']}.txt"
        file_path = os.path.join(output_dir, file_name)
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(chunk['text'])
        print(f"Saved chunk to: {file_path}")

# Process PDFs and prepare for RAG
output_path = "./content_10/chunks-2/"
pdf_files = [
    os.path.join("./content_10/", f)
    for f in os.listdir("./content_10/")
    if f.lower().endswith(".pdf")
]

all_chunks = []
for file_path in pdf_files:
    print(f"\nProcessing: {file_path}")
    
    # Extract text
    pdf_document = Document(document_path=file_path, language='ben')
    pdf2text = PDF2Text(document=pdf_document)
    extracted_content = pdf2text.extract()
    
    # Apply paragraph-based chunking
    chunks = chunk_by_paragraph(
        extracted_content,
        max_characters=1000,       # Max chunk size
        min_characters=200,       # Min chunk size to avoid tiny chunks
        overlap_characters=100    # Overlap to preserve context
    )
    
    # Save chunks
    save_chunks_to_files(chunks, output_path)
    all_chunks.extend(chunks)

print(f"\nTotal chunks extracted from all PDFs: {len(all_chunks)}")

# Example: Prepare chunks for RAG (placeholder for embedding and storage)
for chunk in all_chunks:
    print(f"Chunk ID: {chunk['chunk_id']}")
    print(f"Text (first 100 chars): {chunk['text'][:100]}...")


INFO:multilingual_pdf2text.doc2img.parse_document:Parsing document from pdf to image



Processing: ./content_10/HSC26-Bangla1st-Paper.pdf


INFO:multilingual_pdf2text.ocr.image_to_text:Extracting text from images via OCR


Saved chunk to: ./content_10/chunks-3/chunk_1.txt
Saved chunk to: ./content_10/chunks-3/chunk_2.txt
Saved chunk to: ./content_10/chunks-3/chunk_3.txt
Saved chunk to: ./content_10/chunks-3/chunk_4.txt
Saved chunk to: ./content_10/chunks-3/chunk_5.txt
Saved chunk to: ./content_10/chunks-3/chunk_6.txt
Saved chunk to: ./content_10/chunks-3/chunk_7.txt
Saved chunk to: ./content_10/chunks-3/chunk_8.txt
Saved chunk to: ./content_10/chunks-3/chunk_9.txt
Saved chunk to: ./content_10/chunks-3/chunk_10.txt
Saved chunk to: ./content_10/chunks-3/chunk_11.txt
Saved chunk to: ./content_10/chunks-3/chunk_12.txt
Saved chunk to: ./content_10/chunks-3/chunk_13.txt
Saved chunk to: ./content_10/chunks-3/chunk_14.txt
Saved chunk to: ./content_10/chunks-3/chunk_15.txt
Saved chunk to: ./content_10/chunks-3/chunk_16.txt
Saved chunk to: ./content_10/chunks-3/chunk_17.txt
Saved chunk to: ./content_10/chunks-3/chunk_18.txt
Saved chunk to: ./content_10/chunks-3/chunk_19.txt
Saved chunk to: ./content_10/chunks-3/ch

In [2]:
import os
from unstructured.partition.pdf import partition_pdf

# Directory containing PDFs
output_path = "./content_10/"

# Get list of all PDF files in the directory
pdf_files = [
    os.path.join(output_path, f)
    for f in os.listdir(output_path)
    if f.lower().endswith(".pdf")
]

all_chunks = []

# Process each PDF
for file_path in pdf_files:
    print(f"Processing: {file_path}")
    
    chunks = partition_pdf(
        filename=file_path,
        infer_table_structure=True,  # Extract tables
        strategy="hi_res",           # Mandatory for table extraction
        extract_image_block_types=["Image"],  # Corrected syntax: list of strings, not a single comma-separated string
        extract_image_block_to_payload=True,           # Extract image payloads
        extract_image_block_output_dir=output_path,    # Where to save extracted images
        chunking_strategy="by_title",
        max_characters=10000,
        combine_text_under_n_chars=2000,
        new_after_n_chars=6000,
    )

    all_chunks.extend(chunks)  # Collect chunks from all PDFs
    break

print(f"\nTotal chunks extracted from all PDFs: {len(all_chunks)}")

Processing: ./content_10/HSC26-Bangla1st-Paper.pdf


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]


Total chunks extracted from all PDFs: 18


In [112]:
import os
import time
import re
from langchain_mistralai import ChatMistralAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
os.environ["MISTRAL_API_KEY"] = "RyYQNtsLP4ECwsPjkJBIls9vsgd04AwY" # Make sure .env contains this

# === Configuration ===
DIRECTORY_PATH = "./content_10/chunks-2/"   # Directory containing chunked text files
OUTPUT_DIR = "./content_10/summaries-3/"    # Directory to save summaries
MAX_RETRIES = 10                            # Max retry attempts for API calls
DELAY = 5                                   # Delay between retries (seconds)
START_INDEX = 1                             # Start processing from this chunk index (1-based)

# === Helper: Extract numeric index from chunk filename ===
def extract_chunk_number(filename):
    match = re.search(r"chunk_(\d+)\.txt", filename)
    return int(match.group(1)) if match else float('inf')

# === Load and sort chunk files ===
chunk_files = sorted(
    [f for f in os.listdir(DIRECTORY_PATH) if f.startswith("chunk_") and f.endswith(".txt")],
    key=extract_chunk_number
)

# Ensure START_INDEX is valid
if START_INDEX > len(chunk_files):
    print(f"Error: START_INDEX {START_INDEX} exceeds number of available chunks ({len(chunk_files)}).")
    exit()

# === Prompt Template ===
prompt_text = """
তুমি একজন সহকারী, যার কাজ হলো দশম শ্রেণির বাংলা পাঠ্যবইয়ের গল্প অংশ বিশ্লেষণ করে বিস্তারিত ও অর্থবহ আউটপুট তৈরি করা।

তোমার আউটপুট শুধুমাত্র তখনই থাকবে, যখন পাঠ্যাংশটি **গল্পভিত্তিক বা বিবরণধর্মী** হবে — যেমনঃ চরিত্রের কথা, জীবনের ঘটনা, সংলাপ, বর্ণনা ইত্যাদি।

যদি পাঠ্যাংশে শুধুই **বহুনির্বাচনী প্রশ্ন, শিখনফল, ব্যাখ্যাহীন তথ্য, মূল্যায়ন প্রশ্ন** থাকে এবং কোন গল্প বা ঘটনাবহুল বর্ণনা না থাকে, তখন শুধুই এই বাক্যটি লিখবে:
**"সারাংশ দেওয়া সম্ভব নয়, কারণ এটি গল্প বা বর্ণনামূলক অংশ নয়।"**

**কোনো কিছু অনুমান করবে না এবং যে তথ্য নেই তা লিখবে না।**
**MCQ প্রশ্নের সঠিক উত্তর যদি স্পষ্টভাবে লেখা না থাকে, তা অনুমান করে লিখবে না।**

শুধু নিচের পাঠ্যাংশ ও তার বিশ্লেষণ দাও। কোনো অতিরিক্ত মন্তব্য, ভূমিকা বা অনুবাদ করো না।

---

নির্দেশনা অনুযায়ী আউটপুটের কাঠামো হবে:

পাঠ্যাংশ: 
{element}

**সারাংশ:**
(মূল গল্প অংশ থেকে ৩-৫টি বাক্যে গুরুত্বপূর্ণ সারাংশ লিখো)

**মূল উপাদানসমূহ:**
- প্রধান ঘটনা:
- চরিত্র:
- স্থান ও সময়:
- মূল বার্তা:

**শব্দার্থ ও টীকা (যদি থাকে):**
(গল্প অংশে যে শব্দের অর্থ বা টীকা উল্লেখ আছে, সেগুলো এখানে লিখো)

**প্রশ্ন (ঐচ্ছিক):**
(প্রাসঙ্গিক বিশ্লেষণমূলক বা উন্মুক্ত প্রশ্ন থাকলে এখানে উল্লেখ করো)
"""


prompt = ChatPromptTemplate.from_template(prompt_text)

# === Model Setup ===
# model = ChatMistralAI(
#     model="mistral-large-latest",
#     temperature=0.2,
# )
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
)
summary_chain = prompt | model | StrOutputParser()

# === Create output directory ===
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Summarization loop ===
chunk_summaries = []

for i, filename in enumerate(chunk_files[START_INDEX - 1:], start=START_INDEX):
    file_path = os.path.join(DIRECTORY_PATH, filename)
    with open(file_path, "r", encoding="utf-8") as file:
        chunk_text = file.read()

    retries = 0
    while retries < MAX_RETRIES:
        try:
            summary = summary_chain.invoke(chunk_text)
            chunk_summaries.append((filename, summary))

            # Save the summary
            summary_filename = f"summary_chunk_{i:04d}.txt"
            summary_path = os.path.join(OUTPUT_DIR, summary_filename)
            with open(summary_path, "w", encoding="utf-8") as f:
                f.write(summary)

            print(f"[✓] Chunk {i}/{len(chunk_files)} ({filename}) summarized and saved.")
            time.sleep(DELAY)
            break

        except Exception as e:
            retries += 1
            print(f"[!] Error summarizing chunk {i} ({filename}): {e}")
            if retries < MAX_RETRIES:
                print(f"    Retrying in {DELAY} seconds... (Attempt {retries}/{MAX_RETRIES})")
                time.sleep(DELAY)
            else:
                print(f"    Max retries reached. Skipping chunk {i}.")

print(f"\n✅ Total chunks summarized: {len(chunk_summaries)}")

# Optional: Save mapping of chunk file to summary file
mapping_path = os.path.join(OUTPUT_DIR, "chunk_to_summary_mapping.txt")
with open(mapping_path, "w", encoding="utf-8") as map_file:
    for i, (chunk_name, _) in enumerate(chunk_summaries, start=START_INDEX):
        summary_file = f"summary_chunk_{i:04d}.txt"
        map_file.write(f"{chunk_name} → {summary_file}\n")

print(f"🗂 Mapping saved at: {mapping_path}")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[✓] Chunk 1/48 (chunk_1.txt) summarized and saved.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[✓] Chunk 2/48 (chunk_2.txt) summarized and saved.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

[✓] Chunk 46/48 (chunk_46.txt) summarized and saved.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[✓] Chunk 47/48 (chunk_47.txt) summarized and saved.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[✓] Chunk 48/48 (chunk_48.txt) summarized and saved.

✅ Total chunks summarized: 48
🗂 Mapping saved at: ./content_10/summaries-3/chunk_to_summary_mapping.txt


In [ ]:
# import os
# import time
# from langchain_mistralai import ChatMistralAI
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser
# from dotenv import load_dotenv

# # Load environment variables
# load_dotenv()

# # === Configuration ===
# DIRECTORY_PATH = "./content_10/chunks-2/"  # Directory containing chunked text files
# OUTPUT_DIR = "./content_10/summaries-2/"    # Directory to save summaries
# MAX_RETRIES = 10                          # Max retry attempts for API calls
# DELAY = 5                                 # Delay between retries (seconds)
# START_INDEX = 1                           # Start processing from this chunk index

# # === Load Input Files ===
# text_chunks = []
# chunk_files = sorted(
#     [f for f in os.listdir(DIRECTORY_PATH) if f.startswith("chunk_") and f.endswith(".txt")]
# )

# # Ensure there are enough files
# if START_INDEX >= len(chunk_files):
#     print(f"Error: START_INDEX {START_INDEX + 1} exceeds the number of available files ({len(chunk_files)}).")
#     exit()

# # Load chunk files starting from START_INDEX
# for filename in chunk_files[START_INDEX:]:
#     file_path = os.path.join(DIRECTORY_PATH, filename)
#     with open(file_path, "r", encoding="utf-8") as file:
#         content = file.read()
#         text_chunks.append(content)

# print(f"Loaded {len(text_chunks)} text chunks starting from chunk {START_INDEX + 1} in {DIRECTORY_PATH}.")

# # === Prompt Template ===
# prompt_text = """
# তুমি একজন সহকারী, যার কাজ হলো দশম শ্রেণির বাংলা পাঠ্যবইয়ের গল্প অংশ বিশ্লেষণ করে তথ্যবহুল আউটপুট তৈরি করা।

# তিন ধরনের ইনপুট আসতে পারে:
# 1. **গল্প বা বর্ণনামূলক পাঠ্যাংশ** → তখন সারাংশ তৈরি করবে।
# 2. **MCQ (বহুনির্বাচনী প্রশ্ন)** → যেমন আছে, তেমন রেখে দেবে। কোনো ব্যাখ্যা বা পরিবর্তন করো না।
# 3. **শব্দার্থ/শব্দের অর্থ ও ব্যাখ্যা** → প্রত্যেকটি শব্দ ও তার অর্থ এবং ব্যাখ্যা আলাদা করে উপস্থাপন করো।

# নিচের অংশ বিশ্লেষণ করে উপযুক্ত আউটপুট দাও:

# পাঠ্যাংশ: {element}

# আউটপুট:
# """

# prompt = ChatPromptTemplate.from_template(prompt_text)

# # === Model Setup ===
# os.environ["MISTRAL_API_KEY"] = "RyYQNtsLP4ECwsPjkJBIls9vsgd04AwY"  # Ensure this is set in your .env file
# model = ChatMistralAI(
#     model="mistral-large-latest",
#     temperature=0.2,
# )

# summary_chain = prompt | model | StrOutputParser()

# # === Summarization and File Writing ===
# os.makedirs(OUTPUT_DIR, exist_ok=True)
# chunk_summaries = []

# for i, chunk_text in enumerate(text_chunks):
#     retries = 0
#     chunk_number = START_INDEX + i + 1  # Adjust chunk number for user-friendly output
#     while retries < MAX_RETRIES:
#         try:
#             summary = summary_chain.invoke(chunk_text)
#             chunk_summaries.append(summary)
#             file_name = f"summary_chunk_{chunk_number:04d}.txt"
#             file_path = os.path.join(OUTPUT_DIR, file_name)
#             with open(file_path, "w", encoding="utf-8") as f:
#                 f.write(summary)
#             print(f"Chunk {chunk_number}/{len(chunk_files)} summarized and saved successfully.")
#             time.sleep(DELAY)
#             break
#         except Exception as e:
#             retries += 1
#             print(f"Error summarizing chunk {chunk_number}: {e}")
#             if retries < MAX_RETRIES:
#                 print(f"Retrying in {DELAY} seconds... (Attempt {retries + 1}/{MAX_RETRIES})")
#                 time.sleep(DELAY)
#             else:
#                 print(f"Max retries reached for chunk {chunk_number}. Skipping.")

# print(f"\nTotal chunks summarized: {len(chunk_summaries)}")

Loaded 48 text chunks starting from chunk 1 in ./content_10/chunks-2/.


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Chunk 1/48 summarized and saved successfully.


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Chunk 2/48 summarized and saved successfully.


...

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Chunk 11/48 summarized and saved successfully.


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


Chunk 12/48 summarized and saved successfully.


KeyboardInterrupt: 

In [113]:
import uuid
from base64 import b64decode
from langchain.vectorstores import Chroma, FAISS
from langchain.schema import Document
from langchain.storage import InMemoryStore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.storage import LocalFileStore
import json
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
from typing import Dict, List, Optional, Tuple, Union
from langchain.docstore.base import Docstore, AddableMixin
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from nltk.tokenize import word_tokenize

In [114]:
import os
import json
import uuid
import faiss
from typing import Dict, List, Optional, Tuple, Union
from langchain_openai import ChatOpenAI
from base64 import b64decode
from nltk.tokenize import word_tokenize
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMListwiseRerank
from langchain.retrievers import BM25Retriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.base import Docstore, AddableMixin
import shutil
import tiktoken
import re

# === ADDED FOR TITLE AND QA ===
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

# === Configuration ===
FAISS_DIR = "./project-10-jul-23/"
FAISS_INDEX_PATH = os.path.join(FAISS_DIR, "faiss_index")
DOCSTORE_DIR = os.path.join(FAISS_DIR, "docstore")
BM25_CHUNKS_PATH = os.path.join(FAISS_DIR, "bm25_chunks.json")
BM25_SUMMARIES_PATH = os.path.join(FAISS_DIR, "bm25_summaries.json")
id_key = "doc_id"

os.makedirs(DOCSTORE_DIR, exist_ok=True)

def read_text_files(directory_path: str, prefix: str = "", suffix: str = ".txt") -> List[str]:
    files = sorted([
        f for f in os.listdir(directory_path) if f.startswith(prefix) and f.endswith(suffix)
    ])
    return [
        open(os.path.join(directory_path, filename), encoding="utf-8").read()
        for filename in files
    ]

# === Load Datasets ===
text_raw = read_text_files("/home/rudra/Desktop/Multimodal rag/content_10/chunks-2", "chunk_")
text_summaries = read_text_files("/home/rudra/Desktop/Multimodal rag/content_10/summaries-2", "summary_")
print(":white_check_mark: Datasets loaded.")

# === Docstore ===
class DocumentLocalFileStore(LocalFileStore):
    def mset(self, key_value_pairs):
        serialized_pairs = [
            (key, json.dumps({"page_content": doc.page_content, "metadata": doc.metadata}).encode("utf-8"))
            for key, doc in key_value_pairs
        ]
        super().mset(serialized_pairs)

    def mget(self, keys):
        byte_values = super().mget(keys)
        return [
            None if val is None else Document(**json.loads(val.decode("utf-8")))
            for val in byte_values
        ]

class PersistentDocstore(Docstore, AddableMixin):
    def __init__(self, file_store: DocumentLocalFileStore):
        self.file_store = file_store

    def add(self, texts: Dict[str, Document]) -> None:
        self.file_store.mset(list(texts.items()))

    def delete(self, ids: List[str]) -> None:
        for id in ids:
            self.file_store.delete(id)

    def search(self, search: str) -> Union[str, Document]:
        result = self.file_store.mget([search])
        return result[0] if result[0] is not None else f"Document with ID {search} not found."

    def mget(self, keys: List[str]) -> List[Optional[Document]]:
        return self.file_store.mget(keys)

    def mset(self, key_value_pairs: List[Tuple[str, Document]]) -> None:
        self.file_store.mset(key_value_pairs)

# === Embeddings and VectorStore ===
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
dim = len(embeddings.embed_query("অনুপমের"))
index = faiss.IndexFlatL2(dim)
vectorstore = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=PersistentDocstore(DocumentLocalFileStore(DOCSTORE_DIR)),
    index_to_docstore_id={}
)

# === Retrievers ===
store = DocumentLocalFileStore(DOCSTORE_DIR)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

mv_retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

pd_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    id_key=id_key,
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
_filter = LLMListwiseRerank.from_llm(llm, top_n=1)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=mv_retriever
)

# === Modern prompt with structured JSON output ===
title_qa_prompt = ChatPromptTemplate.from_template("""
You are a careful and factual AI assistant in a multilingual RAG system.

Your job is to extract metadata **strictly from the given content**, which may be in English, Bengali, or both.

You must generate a valid JSON object with the following fields:
- "title": A short, factual English title summarizing the content.
- "questionanswer": A list of 4 question-answer pairs, grounded only in the content.

Instructions:
- Do **not** infer or assume anything not explicitly stated.
- Use only the information **present in the content**.
- If you cannot find 4 valid QA pairs, return only those justified by the content.
- A question and its answer can be in either English or Bengali, depending on the language of the content.
- Do **not translate** the content unless necessary for clarity.
- Your answers must be **verbatim or close paraphrases**, and all facts must be accurate.

Output format (strict JSON, no markdown):

{{
  "title": "Short English title summarizing the content",
  "questionanswer": [
    {{
      "question": "Your question in English or Bengali",
      "answer": "The answer, directly based on the content"
    }},
    ...
  ]
}}
Example Output Format:
{{
  "title": "Anupam's Reflections on Life and Social Pressure",
  "questionanswer": [
    {{
      "question": "অনুপম তার জীবন সম্পর্কে কী ভাবেন?",
      "answer": "তিনি মনে করেন তার জীবন খুব ছোট এবং গুণহীন, তবে এক সময় তিনি ভালোবাসা পেয়েছিলেন বলেই তা মূল্যবান।"
    }},
    {{
      "question": "Who raised Anupam after his father passed away?",
      "answer": "His mother and then his maternal uncle."
    }},
    {{
      "question": "অনুপমের কাকা কেমন ধরনের কনে পছন্দ করতেন?",
      "answer": "যারা ধনী নয় কিন্তু কিছু পণ দেবে, এমন কনে পছন্দ করতেন।"
    }},
    {{
      "question": "How is Anupam's personality described?",
      "answer": "He is quiet, obedient, and conforms to expectations."
    }}
  ]
}}
Content:
{content}
""")


# === Use LangChain 0.1+ syntax (replaces LLMChain) ===
title_qa_chain = title_qa_prompt | llm | StrOutputParser()

def extract_json_string(text: str) -> str:
    """Try to extract the first valid JSON object from text using regex."""
    match = re.search(r"\{.*\}", text, re.DOTALL)
    return match.group(0) if match else ""

def generate_title_and_qa(content: str) -> Tuple[str, List[Dict[str, str]]]:
    try:
        response = title_qa_chain.invoke({"content": content}).strip()
        json_str = response if response.startswith("{") else extract_json_string(response)
        if not json_str:
            raise ValueError("No valid JSON detected in output.")
        parsed = json.loads(json_str)
        title = parsed.get("title", "Untitled")
        print(f"\nGenerated title: {title}")
        qas = parsed.get("questionanswer", [])
        print(f"Generated qas: {qas}.")
        return title, qas
    except Exception as e:
        print(f"\nLLM failed: {e}")
        print("Response:\n", response if 'response' in locals() else "No response")
        print("Content snippet:\n", content[:300])
        return "Untitled", []

# === Utils ===
def save_documents_to_json(documents, path):
    serializable = [
        {"page_content": doc.page_content, "metadata": doc.metadata}
        for doc in documents
    ]
    with open(path, "w") as f:
        json.dump(serializable, f)

def load_documents_from_json(path):
    with open(path, "r") as f:
        raw = json.load(f)
    return [Document(**doc) for doc in raw]

def count_tokens(text: str, model: str = "text-embedding-3-large") -> int:
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

def batch_documents(documents: List[Document], max_tokens: int = 280000):
    batches = []
    current_batch = []
    current_tokens = 0
    for doc in documents:
        tokens = count_tokens(doc.page_content)
        if current_tokens + tokens > max_tokens:
            batches.append(current_batch)
            current_batch = []
            current_tokens = 0
        current_batch.append(doc)
        current_tokens += tokens
    if current_batch:
        batches.append(current_batch)
    return batches

# === Main Logic ===

print("Building new retrievers from data...")


all_ids = [str(uuid.uuid4()) for _ in range(len(text_raw) + len(text_summaries))]
doc_ids = all_ids[:len(text_raw)]
raw_ids = all_ids[len(text_raw):]

# === Enrich Text Summaries ===
summary_texts = []
for i, txt_summary in enumerate(text_summaries):
    title, qas = generate_title_and_qa(txt_summary)
    metadata = {id_key: doc_ids[i], "title": title, "questionanswer": qas}
    summary_texts.append(Document(page_content=txt_summary, metadata=metadata))
mv_retriever.vectorstore.add_documents(summary_texts)
pd_retriever.vectorstore.add_documents(summary_texts)

text_docs = [Document(page_content=text_raw[i], metadata={id_key: doc_ids[i]}) for i in range(len(text_raw))]
mv_retriever.docstore.mset([(doc.metadata[id_key], doc) for doc in text_docs])
pd_retriever.docstore.mset([(doc.metadata[id_key], doc) for doc in text_docs])

# === Prepare Raw Documents ===
all_raw_docs = []
for i, txt_raw in enumerate(text_raw):
    # Generate title and QA pairs for raw text
    title, qas = generate_title_and_qa(txt_raw)
    # Create metadata with raw IDs  
    metadata = {id_key: raw_ids[i], "source": "text", "title": title, "questionanswer": qas}
    all_raw_docs.append(Document(page_content=txt_raw, metadata=metadata))
mv_retriever.vectorstore.add_documents(all_raw_docs)
pd_retriever.vectorstore.add_documents(all_raw_docs)
# Save raw documents to docstore
mv_retriever.docstore.mset([(doc.metadata[id_key], doc) for doc in all_raw_docs])
pd_retriever.docstore.mset([(doc.metadata[id_key], doc) for doc in all_raw_docs])



# === Prepare BM25 ===
all_raw_docs_bm25 = [Document(page_content=s.page_content, metadata={"source": "text"}) for s in text_docs]
save_documents_to_json(all_raw_docs_bm25, BM25_CHUNKS_PATH)
all_summary_texts = [Document(page_content=s.page_content, metadata={"source": "text"}) for s in summary_texts]
save_documents_to_json(all_summary_texts, BM25_SUMMARIES_PATH)

bm25_retriever = BM25Retriever.from_documents(all_raw_docs_bm25)
bm25_retriever.k = 5
bm25_docs_retriever = BM25Retriever.from_documents(all_summary_texts)
bm25_docs_retriever.k = 5


vectorstore.save_local(f"{FAISS_DIR}faiss_index")
print(":white_check_mark: Retrievers initialized with enriched metadata.")


:white_check_mark: Datasets loaded.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Building new retrievers from data...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Generated title: Social Identity Crisis and Human Values in 'অপরিচিতা'
Generated qas: [{'question': 'নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে কী সংকট সৃষ্টি হয়?', 'answer': 'সমাজে পরিচয় সংকট সৃষ্টি হয়।'}, {'question': 'অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?', 'answer': 'ডাক্তারি, ওকালতি, মাস্টারি, বা ব্যবসা।'}, {'question': "দীপুর চাচার সঙ্গে “অপরিচিতা' গল্পের কোন চরিত্রের মিল আছে?", 'answer': 'মামার।'}, {'question': 'অনুপমের বয়স কত বছর?', 'answer': 'পঁচিশ, ছাব্বিশ, সাতাশ, বা আটাশ।'}].


...


Generated title: Wedding Jewelry Inspection and Family Dynamics
Generated qas: [{'question': 'মামা বিবাহ-বাড়িতে কেন খুশি হননি?', 'answer': 'উঠানটাতে বরযাত্রীদের জায়গা সংকুলান হওয়াই শক্ত এবং সমস্ত আয়োজন নিতান্ত মধ্যম রকমের।'}, {'question': 'শস্তুনাথবাবু কি করতে চান?', 'answer': 'তিনি কনের সমস্ত গহনা যাচাই করিয়া দেখিবেন।'}, {'question': 'অনুপমের প্রতিক্রিয়া কী ছিল যখন শস্তুনাথবাবু তাকে ডাকলেন?', 'answer': 'তিনি মাথা হেট করিয়া চুপ করিয়া রহিলেন।'}, {'question': 'গহনা কেমন ছিল যা শস্তুনাথবাবু আনলেন?', 'answer': 'সকল গহনা পিতামহীদের আমলের ছিল, হাল ফ্যাশনের সূম্ষ্ম কাজ নয়।'}].


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Generated title: Discussion on Jewelry and Wedding Arrangements
Generated qas: [{'question': 'মামা গহনা নিয়ে কী বললেন?', 'answer': "তিনি বললেন, 'এ আর দেখিব কী। ইহাতে খাদ নাই-এমন সোনা এখনকার দিনে ব্যবহারই হয় না।'"}, {'question': 'শস্তুনাথবাবু বরযাত্রীদের খাওয়ানোর পর কী বললেন?', 'answer': "তিনি বললেন, 'আপনাদিগকে অনেক কষ্ট দিয়াছি। আমরা ধনী নই, আপনাদের যোগ্য আয়োজন করিতে পারি নাই, ক্ষমা করিবেন।'"}, {'question': 'মামা কেন খেতে বসতে পারলেন না?', 'answer': 'মামার বিরুদ্ধে চলা আমার পক্ষে অসম্ভব।'}, {'question': 'শস্তুনাথবাবু কেমন ধরনের মানুষ হিসেবে বর্ণিত হলেন?', 'answer': 'লোকটি নেহাত ভালোমানুষ-ধরনের, কিন্তু ভিতরে বেশ একটু জোর আছে বলিয়া বোধ হইল।'}].


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


:white_check_mark: Retrievers initialized with enriched metadata.


In [115]:
mv_retriever.invoke(" অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(metadata={'doc_id': 'ccc4052d-ba6e-4916-9012-3ac65f202107', 'source': 'text', 'title': "Anupam's Family Dynamics and Societal Pressures", 'questionanswer': [{'question': 'অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?', 'answer': 'ওকালতি'}, {'question': 'মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ কী?', 'answer': 'তার প্রভাব'}, {'question': "দীপুর চাচার সঙ্গে “অপরিচিতা' গল্পের কোন চরিত্রের মিল আছে?", 'answer': 'মামার'}, {'question': 'দীপুর চাচার চরিত্রে কোন বৈশিষ্ট্য প্রাধান্য পেয়েছে?', 'answer': 'লোভ'}]}, page_content='্রের\nঅমানবিকতার স্ফুরণ যেমনঘটেছে, তেমনি একই সঙ্গে পুরুষের ভাষ্যে নারীর প্রশস্তিও কীর্তিত হয়েছে।\n\n19 [15026]\nঅনলাইন ব্যাট" 10\n10401০:\n\nপাঠ্যপুস্তকের প্রশ্ন\nবহুনির্বাচনী\n\n১। অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?\n(ক) ডাক্তারি (খ) ওকালতি (গ) মাস্টারি (ঘ) ব্যবসা উত্তর: খ\n২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার-\n(ক) প্রতিপত্তি (খ) প্রভাব (গ) বিচক্ষণতা (ঘ) কুট বুদ্ধি উত্তর: খ\n৩। দীপুর চাচার সঙ্গে “অপরিচিতা\' গল্পের কোন চরিত্রের মিল আছে?\n(ক

In [116]:
pd_retriever.invoke("Anupam?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(metadata={'doc_id': '5bcb99d1-d5fe-4252-8c7a-89d0a0a5a7f9'}, page_content="ো ভাই। তাহার\nমতো রুচি এবং দক্ষতার “পরে আমি ষোলো-আনা নির্ভর করিতে পারি। বিনুদা ফিরিয়া আসিয়া বলিলেন, 75026\n19414\n“মন্দ নয় হে! খাটি সোনা বটে!” বিনুদাদার ভাষাটা অত্যন্ত আঁট। যেখানে আমরা বলি 'চমৎকার' সেখানে তিনি\nবলেন “চলনসই'। অতএব বুঝিলাম, আমার ভাগ্যে প্রজাপতির সঙ্গে পঞ্চশরের কোনো বিরোধ নাই।\nবলা বাহুল্য, বিবাহ-উপলক্ষে কন্যাপক্ষকেই\nকলিকাতা আসিতে হইল। কন্যার পিতা শ্তুনাথবাবু হরিশকে কত বিশ্বাস করেন তাহার প্রমাণ এই যে, বিবাহের\nতিন দিন পূর্বে তিনি আমাকে চক্ষে দেখেন এবং আশীর্বাদ করিয়া যান। বয়স তার চল্সিশের কিছু এপারে বা\nওপারে। চুল কাচা, গোফে পাক ধরিতে আরম্ভ করিয়াছে মাত্র। সুপুরুষ বটে। ভিড়ের মধ্যে দেখিলে সকলের\nআগে তার উপরে চোখ পড়িবার মতো চেহারা।\nআশা করি আমাকে দেখিয়া তিনি খুশি হইয়াছিলেন। বোঝা শক্ত, কেননা তিনি বড়ই চুপচাপ। যে দুটি-একটি\nকথা বলেন যেন তাহাতে পুরা জোর দিয়া বলেন না। মামার মুখ তখন অনর্গল ছুটিতেছিল- ধনে মানে আমাদের\nস্থান যে শহরের কারও চেয়ে কম নয়, সেইটেকেই তিনি নানা প্রসঙ্গে প্রচার 

In [117]:
import json
from base64 import b64decode

from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI

# Optional: for language detection (pip install langdetect)
from langdetect import detect


In [118]:
def optimize_query(query: str) -> dict:
    """
    Use LLM to generate optimized multilingual queries for both vector and BM25 retrievers.
    
    Input: 
        query (str): Original user query in Bengali or English.
    Returns:
        dict: {
            "vector_query": semantically rich query for vector retriever,
            "bm25_keywords": keyword-only version for BM25 retriever
        }
    """
    system_prompt = """
You are an expert in multilingual query optimization for a RAG (Retrieval-Augmented Generation) pipeline that supports both English and Bengali (Bangla).

Your job is to:
1. Reformulate the original question clearly and naturally in its own language (English or Bengali) for use with a vector-based retriever.
2. Extract concise and essential keywords (nouns, names, concepts) for use with a BM25 retriever. Only include important terms, and return them as a space-separated string — do not translate the language.

Output JSON must follow this format:
{
  "vector_query": "rephrased full question (same language)",
  "bm25_keywords": "keyword1 keyword2 keyword3"
}
Only return valid JSON.
"""

    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_prompt.strip()),
        HumanMessage(content=f"Original Query: {query}")
    ])

    llm = ChatOpenAI(model="gpt-4o-mini")  # or gpt-4, mistral if multilingual
    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({"query": query})
    
    try:
        result = json.loads(response)
        return {
            "vector_query": result.get("vector_query", query).strip(),
            "bm25_keywords": result.get("bm25_keywords", query).strip()
        }
    except json.JSONDecodeError:
        print("[Warning] LLM did not return valid JSON. Falling back to original query.")
        return {
            "vector_query": query,
            "bm25_keywords": query
        }


In [119]:
def combine_retrievers(query):
    optimized_queries = optimize_query(query)
    vector_query = optimized_queries["vector_query"]
    bm25_keywords = optimized_queries["bm25_keywords"]

    mv_docs = mv_retriever.invoke(vector_query)
    pd_docs = pd_retriever.invoke(vector_query)
    bm25_docs = bm25_retriever.invoke(bm25_keywords)
    bm25_summary_docs = bm25_docs_retriever.invoke(bm25_keywords)
    print(f"Docs: {mv_docs}, {pd_docs}, {bm25_docs}, {bm25_summary_docs}")
    seen = set()
    combined_docs = []
    for doc in mv_docs + pd_docs + bm25_docs + bm25_summary_docs:
        if doc.page_content not in seen:
            seen.add(doc.page_content)
            combined_docs.append(doc)

    return combined_docs


In [121]:
def parse_docs(docs):
    """Split base64-encoded images and plain text."""
    b64 = []
    text = []
    for doc in docs:
        if not isinstance(doc, Document):
            print(f"Skipping non-document: {type(doc)}")
            continue
        try:
            b64decode(doc.page_content)
            b64.append(doc.page_content)
        except Exception:
            text.append(doc)
    return {"images": b64, "texts": text}


In [122]:
def detect_lang(text):
    try:
        return detect(text)
    except:
        return "en"


In [123]:
def build_prompt(kwargs):
    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]
    lang = detect_lang(user_question)

    context_text = ""
    for text_element in docs_by_type["texts"]:
        source = text_element.metadata.get("source", "unknown")
        context_text += f"[Source: {source}]\n{text_element.page_content}\n\n"

    prompt_template = f"""
You are an AI assistant answering questions in Bengali or English based on context from textbooks or official documents.

- Use only the most relevant parts of the context to answer.
- Maintain structure and accuracy when referring to tables, names, or numbers.
- Ignore irrelevant or duplicate context.
- Respond in the same language as the user question ("{lang}").

Context:
{context_text}

Question:
{user_question}
"""

    prompt_content = [{"type": "text", "text": prompt_template}]

    return ChatPromptTemplate.from_messages([
        HumanMessage(content=prompt_content)
    ])


In [124]:
llm = ChatOpenAI(model="gpt-4o-mini")

chain = (
    {
        "context": RunnableLambda(combine_retrievers) | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | llm
    | StrOutputParser()
)


In [125]:
query = "অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': 'd3dda0c8-8c00-4f62-94d8-b2633a26c978', 'source': 'text', 'title': 'Understanding Social Issues through Online Batch Learning', 'questionanswer': [{'question': 'অনলাইন ব্যাচের মাধ্যমে শিক্ষার্থীরা কী শিখবে?', 'answer': 'নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।'}, {'question': 'দীপুর চাচা কেন বিয়ের আলোচনা ভেঙে দেন?', 'answer': 'যৌতুক নিয়ে বাড়াবাড়ি করার কারণে কন্যার পিতা অপমানিত বোধ করে বিয়ের আলোচনা ভেঙে দেন।'}, {'question': 'অনুপমের বয়স কত বছর?', 'answer': 'ছাব্বিশ।'}, {'question': "দীপুর চাচার সঙ্গে 'অপরিচিতা' গল্পের কোন চরিত্রের মিল আছে?", 'answer': 'হরিশের।'}]}, page_content="অপরিচিতা\n\nঅনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,\n\nকলকরো ৬ 86919 [115026]\nনী '৯1৬]খ7চ\n\nঅনলাইন ব্যাচ\n10401০:\nভ্ঁ শিখনফল\n\n৮ নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।\n\n৮ তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।\n\n৮ তৎকালীন সমাজের পণপ্রথার ক

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: অনুপমের বাবা ওকালতি করে জীবিকা নির্বাহ করতেন।


In [126]:
query = "অনুপমের মামা সম্পর্কে কী কী তথ্য পাওয়া যায়?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '5bcb99d1-d5fe-4252-8c7a-89d0a0a5a7f9'}, page_content="ো ভাই। তাহার\nমতো রুচি এবং দক্ষতার “পরে আমি ষোলো-আনা নির্ভর করিতে পারি। বিনুদা ফিরিয়া আসিয়া বলিলেন, 75026\n19414\n“মন্দ নয় হে! খাটি সোনা বটে!” বিনুদাদার ভাষাটা অত্যন্ত আঁট। যেখানে আমরা বলি 'চমৎকার' সেখানে তিনি\nবলেন “চলনসই'। অতএব বুঝিলাম, আমার ভাগ্যে প্রজাপতির সঙ্গে পঞ্চশরের কোনো বিরোধ নাই।\nবলা বাহুল্য, বিবাহ-উপলক্ষে কন্যাপক্ষকেই\nকলিকাতা আসিতে হইল। কন্যার পিতা শ্তুনাথবাবু হরিশকে কত বিশ্বাস করেন তাহার প্রমাণ এই যে, বিবাহের\nতিন দিন পূর্বে তিনি আমাকে চক্ষে দেখেন এবং আশীর্বাদ করিয়া যান। বয়স তার চল্সিশের কিছু এপারে বা\nওপারে। চুল কাচা, গোফে পাক ধরিতে আরম্ভ করিয়াছে মাত্র। সুপুরুষ বটে। ভিড়ের মধ্যে দেখিলে সকলের\nআগে তার উপরে চোখ পড়িবার মতো চেহারা।\nআশা করি আমাকে দেখিয়া তিনি খুশি হইয়াছিলেন। বোঝা শক্ত, কেননা তিনি বড়ই চুপচাপ। যে দুটি-একটি\nকথা বলেন যেন তাহাতে পুরা জোর দিয়া বলেন না। মামার মুখ তখন অনর্গল ছুটিতেছিল- ধনে মানে আমাদের\nস্থান যে শহরের কারও চেয়ে কম নয়, সেইটেকেই তিনি নানা প্রসঙ্গে প

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: অনুপমের মামা সম্পর্কে নিম্নলিখিত তথ্য পাওয়া যায়:

1. তিনি বিয়েতে নগদ টাকা ও গহনা পণ হিসেবে দাবি করেন।
2. অনুপমের বিয়ের সময় কন্যার বাবার কাছে কনেপক্ষের গহনা যাচাই করার জন্য সেকরা নিয়ে আসেন, যা কনেপক্ষের অপমানজনক আচরণ হিসেবে বিবেচিত হয়।
3. মামার চরিত্র যৌতুকলোভী এবং তিনি অনেক টাকা যৌতুক পাওয়ার জন্য একাধিক মেয়ে খুঁজছিলেন।
4. মামা সব সময় নিজেকে অসামান্য চতুর মনে করেন।
5. তিনি বিয়ের আয়োজন ও মাতৃস্নেহের প্রভাবে অমঙ্গল আনয়ন করেন, যে কারণে গল্পের কেন্দ্রীয় চরিত্র অনুপম সীমাবদ্ধতার মধ্যে থাকে।


In [127]:
query = "What kind of son-in-law does Anupam’s uncle prefer?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '1cd392cd-dfd6-4ab0-b73d-c8757424f6a3', 'source': 'text', 'title': "Questions from Anupam's Marriage Context", 'questionanswer': [{'question': 'বিবাহের কতদিন পূর্বে অনুপমের সাথে তার শ্বশুরের সাক্ষাৎ হয়?', 'answer': '৩ দিন'}, {'question': "তিনি বড়ই চুপচাপ' এখানে কার কথা বলা হয়েছে?", 'answer': 'মামা'}, {'question': 'রেল কর্মচারী কতটি টিকিট বেঞ্চে ঝুলিয়েছিলেন?', 'answer': 'দুইটি'}, {'question': "'তোমার নাম কী?' - কল্যাণীকে কে জিজ্ঞাসা করল?", 'answer': 'অনুপম'}]}, page_content='মন্তী\n\n(ঘ) অনুপমের\n\n(ঘ) বিয়ে উপলক্ষে\n\n(ঘ) শঙ্ুনাথকে\n\n(ঘ) মামার\n\n(ঘ) হরিশের\n\n(ঘ) ভ্রমণের\n\n(ঘ) ১৭ বছর সী\n\n19:45\n৫&৫। মামার বাহিরের যাত্রাপথের সীমানা কতদুর?\n\n(ক) আন্দামান পর্যন্ত (খ) কোন্নগর পর্যন্ত (গ) কানপুর পর্যন্ত (ঘ) হাওড়া পর্যন্ত\n৫৬। বিবাহের কতদিন পূর্বে অনুপমের সাথে তার শ্বশুরের সাক্ষাৎ হয়?\n\n(ক) ২ দিন (খ) ৩ দিন (গ) ৪ দিন (ঘ) ৫ দিন\n\n৫৭। "তিনি বড়ই চুপচাপ\' এখানে কার কথা বলা হয়েছে?\n\n(ক) মামা (খ) হরিশ (গ) শস্তুনাথ (ঘ) মা\n\n৫৮। \'তিনি কিছুতেই ঠকব

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: Anupam's uncle prefers a son-in-law who is not wealthy and would come to their house with humility. He does not like a daughter of a rich family and desires a groom who will not be greedy, indicating a preference for simplicity and modesty in the potential son-in-law.


In [128]:
query = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '921433b7-dd1b-41bb-9e41-be3263e0a178', 'source': 'text', 'title': 'The Impact of Dowry on Women in Society', 'questionanswer': [{'question': 'অনুপমের মামা বিয়েতে কী দাবি করেন?', 'answer': 'বিয়েতে নগদ টাকা ও গহনা পণ হিসেবে দাবি করেন।'}, {'question': 'হারুন মিয়া কেন বিয়ে ভেঙে দিতে চান?', 'answer': 'পণের সামান্য টাকা বাকি থাকায় হারুন মিয়া বিয়ে ভেঙে দিতে চান।'}, {'question': 'যৌতুকপ্রথা সম্পর্কে লেখকের মতামত কী?', 'answer': 'যৌতুকপ্রথা একটি সামাজিক ব্যাধি এবং এটি আমাদের সমাজে ভয়াল রূপ ধারণ করেছে।'}, {'question': 'অনুপমের মামার আচরণে কী প্রকাশ পায়?', 'answer': 'অনুপমের মামার আচরণ ও কথাবার্তায় তার হীনতা, লোভ ও অমানবিকতা প্রকাশ পায়।'}]}, page_content='য নয়।\nসেএকজনকে জীবনসঙ্গী করতে এসেছে, অপমান করতে নয়। \'অপরিচিতা\' গল্পের অনুপমও শিক্ষিত, মার্জিত।\n\n21 [15026]\nঅনলাইন ব্যাট" 10\n10401০:\n\nকিন্তু স্পষ্ট কথা বলার মতো সাহস তার নেই। নিজের সিদ্ধান্ত সে নিজে নিতে পারে না। সেকরা দিয়ে গহনা\nযাচাই যে কনেপক্ষের অপমান তা অনুপম বুঝতে পারে না। এতে তার ব

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: অনুপমের ভাষায় সুপুরুষ বলতে শ্তুনাথ বাবুকে নির্দেশ করা হয়েছে।


In [129]:
query = "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': 'd581bada-2943-4426-a3a2-a96fa774963f'}, page_content='ামার সঙ্গে মা একযোগে বিস্তর হাসিলেন\n(ঘ) ঠাট্টার সম্পর্ককে স্থায়ী করিবার ইচ্ছা আমার নাই উত্তর: ক\n\n26 [নেন\nঅনলাইন ব্যাট" 10\n19475\n\nনিচের উদ্দীপকটি পড়ে ৩৬ ও ৩৭ নম্বর প্রশ্নের উত্তর দাও:\n\nশাওনের বিয়ে চুড়ান্ত হয় অন্যার সাথে। যৌতুকের দাবি পূরণ না হওয়ায় মোতালেব সাহেব ছেলের বিয়ে ভেঙে\nদিতে চান। বাবার অন্যায় আবদার শাওন মানতে নারাজ। সে যুক্তি দিয়ে বাবাকে বুঝিয়ে যৌতুক না নিয়েই\nঅন্যাকে বিয়ে করে। [ব. বো. \'১৯]\n\n৩৬। মোতালেব সাহেব \'অপরিচিতা\' গল্পের কোন চরিত্রের ইঙ্গিতবহ?\n\n(ক) হরিশ (খ) বিনুদা (গ) মামা (ঘ) শস্তুনাথ উত্তর: গ\n৩৭। শাওনের কোন কোন বৈশিষ্ট্য অনুপমের মধ্যে থাকলে অনুপমের বিয়েটা টিকে যেত?\n\n1. সাহসিকতা\n\n1. ব্যক্তিত্ব\n\n11. গভীর ভালোবাসা\n\nনিচের কোনটি সঠিক?\n\n(ক), (খ)।, 11 (গ) |, 1 (ঘ)1, 1111 উত্তর: ক\nনিচের উদ্দীপকটি পড়ে ৩৮ ও ৩৯ নম্বর প্রশ্নের উত্তর দাও:\n\nআদিব ও শাফিক দুই বন্ধু। আবিদ অহংকারী, নিজবি, পৌরুষশূন্য। অন্যদিকে শাফিক উচ্ছল, রসিক।\nশাফিক যেকোনো পরিবেশে দ্রুত

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: অনুপমের ভাগ্য দেবতা হিসাবে উল্লেখ করা হয়েছে মামাকে।


In [130]:
query = "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': 'ce5fb98e-88f1-446b-ac8a-1189136d03bb', 'source': 'text', 'title': 'Questions and Answers from the Content', 'questionanswer': [{'question': "ভাইটি' এখানে 'ছোট - ভাইটি' বলতে কাকে বোঝানো হয়েছে?", 'answer': 'গণেশ'}, {'question': 'স্টেশনে অনুপম কী ফেলে গেল?', 'answer': 'টিকিট'}, {'question': 'বিবাহের সময় অনুপমের বয়স কত ছিল?', 'answer': '২১ বছর'}, {'question': 'কল্যাণী স্টেশন হতে কী খাবার কিনে নেয়?', 'answer': 'ঝালমুড়ি'}]}, page_content="ভাইটি' এখানে 'ছোট - ভাইটি' বলতে কাকে বোঝানো হয়েছে?\n(ক) গণেশ (খ) প্রজাপতি (গ) কার্তিক (ঘ) পঞ্চশর\n\n34 ইন ব্যাচ”\n\n৩৯। 'এখানে জায়গা আছে' উক্তিটি কার?\n\n(ক) আর্দালির (খ) গার্ডের (গ) কল্যাণীর\n৪০। স্টেশনে অনুপম কী ফেলে গেল?\n(ক) টিকিট (খ) ক্যামেরা (গ) তোরঙ্গ\n\n৪১। ট্রেনে দেখা হওয়ার সময় কল্যাণীর বয়স কত ছিল?\n(ক) ১৪/১৫ বছর (খ) ১৫/১৬ বছর (গ) ১৬/১৭ বছর\n৪২। অপরিচিতা মেয়েটির সঙ্গে কতজন মেয়ে ছিল?\n\n(ক) ২/৩ জন (খ) ৩/৪ জন (গ) ৪/৫ জন\n৪৩। কল্যাণী স্টেশন হতে কী খাবার কিনে নেয়?\n\n(ক) চানা-মুঠ (খ) ঝালমুড়ি (গ) চিনেবা

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: বিয়ের সময় কল্যাণীর প্রকৃত বয়স ছিল (ক) ১৪ বছর।


In [131]:
query = "How did Binudada describe the bride’s appearance and character?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': 'f95d512f-441d-441c-b5cb-c74494893cfb'}, page_content='এ গাড়ির এই দুই বেঞ্চ আগে হইতেই দুই সাহেব রিজার্ভ\nকরিয়াছেন, আপনাদিগকে অন্য গাড়িতে যাইতে হইবে।"\n\n15 আমি তো তাড়াতাড়ি ব্যস্ত হইয়া দীড়াইয়া উঠিলাম। মেয়েটি হিন্দিতে বলিল, “না, আমরা গাড়ি ছাড়িব না।”\n\nসে লোকটি রোখ করিয়া বলিল, “না ছাড়িয়া উপায় নাই।”\n\nকিন্তু, মেয়েটির চলিষ্ুতার কোনো লক্ষণ না দেখিয়া সে নামিয়া গিয়া ইংরেজ স্টেশন-মাস্টারকে ডাকিয়া\nআনিল। সে আসিয়া আমাকে বলিল, “আমি দুঃখিত, কিন্তু-”\n\nশুনিয়া আমি “কুলি কুলি\' করিয়া ডাক ছাড়িতে লাগিলাম। মেয়েটি উঠিয়া দুই চক্ষে অগ্নিবর্ষণ করিয়া বলিল,\n“না, আপনি যাইতে পারিবেন না, যেমন আছেন বসিয়া থাকুন।”\n\nবলিয়া সে দ্বারের কাছে দীড়াইয়া স্টেশনমাস্টারকে ইংরেজি ভাষায় বলিল, “এ গাড়ি আগে হইতে রিজার্ভ\nকরা, এ কথা মিথ্যা কথা।” বলিয়া নাম লেখা টিকিটটি খুলিয়া প্ল্যাটফর্মে ছুঁড়িয়া ফেলিয়া দিল।\n\nবলিয়া সে দ্বারের কাছে দীড়াইয়া স্টেশনমাস্টারকে ইংরেজি ভাষায় বলিল, “এ গাড়ি আগে হইতে রিজার্ভ\nকরা, এ কথা মিথ্যা কথা।” বলিয়া নাম লেখা টিকিটটি খুল

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: Binudada described the bride's appearance as extraordinarily beautiful, indicating a deep admiration for her. He seemed to convey that she possessed an enchanting charm, which was evident in their discussions. However, the specific details of her character, besides her beauty, highlighted how her persona had captivated the attention and affection of those around her. Overall, Binudada's description reflects a perspective of allure and fascination towards the bride.


In [132]:
query = "হরিশ কনের বাবার সম্পর্কে কী কী বলেছেন?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '25a205bc-4ab6-4489-bd68-b4b24642df0c', 'source': 'text', 'title': "Anupam's Relationship with His Uncle", 'questionanswer': [{'question': "Who is Anupam's guardian?", 'answer': 'His uncle.'}, {'question': 'অনুপমের মামা কত বছর বড়?', 'answer': 'ছয় বছর।'}, {'question': "What is a characteristic of Anupam's uncle?", 'answer': 'তিনি অনুপমের সংসার আঁকড়ে আছেন।'}, {'question': "What type of son-in-law does Anupam's uncle prefer?", 'answer': 'যার তেজ নেই এবং টাকা দিতে কসুর করবে না।'}]}, page_content='মামাকে ভুলতে দেন না\n\n॥. অনুপমকে ভুলতে দেন না\n\nনিচের কোনটি সঠিক?\n\n(ক), (খ)।, | (গ)।, | (ঘ)।,11, 1\n\n38 [15026]\nঅনলাইন ব্যাট" 10\n10401০:\n\n৯২। কোন তথ্যগুলো অনুপমের মামার ক্ষেত্রে প্রযোজ্য ঃ\n\n1. মামাই অনুপমের অভিভাবক\n\n1॥. তিনি অনুপমের চেয়ে বড়জোর বছর ছয়েকের বড়\n\n1. ফন্ধুর বালির মতো তিনি অনুপমের সংসার আঁকড়ে আছেন\n\nনিচের কোনটি সঠিক?\n\n(ক), (খ)।, | (গ)।, | (ঘ)।,11, 1\n৯৩। মামার পছন্দের বেয়াই এমন-\n\n1. যার তেজ নেই\n\n1. টাকা দিতে কসুর করবে না\

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: হরিশ কনের বাবার সম্পর্কে বলেছেন:

1. এককালে তাদের বংশে লক্ষ্মীর মঙ্গলঘট উপুড় করা ছিল।
2. দেশে বংশমর্যাদা রক্ষা করে চলা কঠিন বলে পশ্চিমে গিয়ে বাস করছেন।
3. কানপুরে তিনি একজন প্রতিষ্ঠিত ডাক্তার।

এই তথ্যগুলি থেকে বোঝা যায় যে, কনের বাবার পূর্বের অবস্থান এবং বর্তমান পরিস্থিতি কেমন।


In [133]:
query = "Who was Anupam’s guardian and how did he influence Anupam’s life decisions?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '5bcb99d1-d5fe-4252-8c7a-89d0a0a5a7f9'}, page_content="ো ভাই। তাহার\nমতো রুচি এবং দক্ষতার “পরে আমি ষোলো-আনা নির্ভর করিতে পারি। বিনুদা ফিরিয়া আসিয়া বলিলেন, 75026\n19414\n“মন্দ নয় হে! খাটি সোনা বটে!” বিনুদাদার ভাষাটা অত্যন্ত আঁট। যেখানে আমরা বলি 'চমৎকার' সেখানে তিনি\nবলেন “চলনসই'। অতএব বুঝিলাম, আমার ভাগ্যে প্রজাপতির সঙ্গে পঞ্চশরের কোনো বিরোধ নাই।\nবলা বাহুল্য, বিবাহ-উপলক্ষে কন্যাপক্ষকেই\nকলিকাতা আসিতে হইল। কন্যার পিতা শ্তুনাথবাবু হরিশকে কত বিশ্বাস করেন তাহার প্রমাণ এই যে, বিবাহের\nতিন দিন পূর্বে তিনি আমাকে চক্ষে দেখেন এবং আশীর্বাদ করিয়া যান। বয়স তার চল্সিশের কিছু এপারে বা\nওপারে। চুল কাচা, গোফে পাক ধরিতে আরম্ভ করিয়াছে মাত্র। সুপুরুষ বটে। ভিড়ের মধ্যে দেখিলে সকলের\nআগে তার উপরে চোখ পড়িবার মতো চেহারা।\nআশা করি আমাকে দেখিয়া তিনি খুশি হইয়াছিলেন। বোঝা শক্ত, কেননা তিনি বড়ই চুপচাপ। যে দুটি-একটি\nকথা বলেন যেন তাহাতে পুরা জোর দিয়া বলেন না। মামার মুখ তখন অনর্গল ছুটিতেছিল- ধনে মানে আমাদের\nস্থান যে শহরের কারও চেয়ে কম নয়, সেইটেকেই তিনি নানা প্রসঙ্গে প

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: Anupam's guardian was his maternal uncle (mama). He played a significant role in influencing Anupam's life decisions, particularly regarding marriage. Anupam's uncle had a specific preference for the kind of bride he wanted, desiring a girl who would come to their household with humility and without a focus on wealth. Despite the fact that Anupam was of marriageable age, his uncle's opinions and expectations greatly restricted Anupam's autonomy, contributing to his lack of assertiveness and personal agency in significant life choices. This resulted in Anupam being portrayed as a character lacking individuality and decisiveness, ultimately leading to the failure of his marriage arrangement.


In [134]:
query = "What do we learn about Anupam’s uncle’s personality during the bride-seeing visit?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '5bcb99d1-d5fe-4252-8c7a-89d0a0a5a7f9'}, page_content="ো ভাই। তাহার\nমতো রুচি এবং দক্ষতার “পরে আমি ষোলো-আনা নির্ভর করিতে পারি। বিনুদা ফিরিয়া আসিয়া বলিলেন, 75026\n19414\n“মন্দ নয় হে! খাটি সোনা বটে!” বিনুদাদার ভাষাটা অত্যন্ত আঁট। যেখানে আমরা বলি 'চমৎকার' সেখানে তিনি\nবলেন “চলনসই'। অতএব বুঝিলাম, আমার ভাগ্যে প্রজাপতির সঙ্গে পঞ্চশরের কোনো বিরোধ নাই।\nবলা বাহুল্য, বিবাহ-উপলক্ষে কন্যাপক্ষকেই\nকলিকাতা আসিতে হইল। কন্যার পিতা শ্তুনাথবাবু হরিশকে কত বিশ্বাস করেন তাহার প্রমাণ এই যে, বিবাহের\nতিন দিন পূর্বে তিনি আমাকে চক্ষে দেখেন এবং আশীর্বাদ করিয়া যান। বয়স তার চল্সিশের কিছু এপারে বা\nওপারে। চুল কাচা, গোফে পাক ধরিতে আরম্ভ করিয়াছে মাত্র। সুপুরুষ বটে। ভিড়ের মধ্যে দেখিলে সকলের\nআগে তার উপরে চোখ পড়িবার মতো চেহারা।\nআশা করি আমাকে দেখিয়া তিনি খুশি হইয়াছিলেন। বোঝা শক্ত, কেননা তিনি বড়ই চুপচাপ। যে দুটি-একটি\nকথা বলেন যেন তাহাতে পুরা জোর দিয়া বলেন না। মামার মুখ তখন অনর্গল ছুটিতেছিল- ধনে মানে আমাদের\nস্থান যে শহরের কারও চেয়ে কম নয়, সেইটেকেই তিনি নানা প্রসঙ্গে প

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: During the bride-seeing visit, Anupam's uncle demonstrates a calculating and somewhat shrewd personality. He does not display much verbal communication and remains quiet, indicating a reserved nature. His satisfaction with the girl’s family background and financial status is clear, as he believes that he must ensure there are no unfair dealings regarding the bride's dowry. His approach to the discussion about dowry shows that he is astute and careful, aiming to secure the best deal possible for their family. Additionally, he exhibits a mix of pride and cunning when he believes he has outsmarted the bride's family during negotiations. Overall, his character reflects traits of careful planning, strategic thinking, and a focus on financial matters.


In [135]:
query = "অনুপমের মামা কেমন জামাই পছন্দ করতেন?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '9315dba7-327b-4c5c-b522-01d39f9abf82', 'source': 'text', 'title': 'Shawon and the Challenges of Dowry in Marriage', 'questionanswer': [{'question': 'শাওনের বিয়ে কেন ভেঙে যেতে চেয়েছিল মোতালেব সাহেব?', 'answer': 'যৌতুকের দাবি পূরণ না হওয়ায় মোতালেব সাহেব ছেলের বিয়ে ভেঙে দিতে চান।'}, {'question': 'শাওন কিভাবে তার বাবাকে বুঝিয়েছিল?', 'answer': 'সে যুক্তি দিয়ে বাবাকে বুঝিয়ে যৌতুক না নিয়েই অন্যাকে বিয়ে করে।'}, {'question': 'আদিব এবং শাফিকের মধ্যে কি পার্থক্য আছে?', 'answer': 'আদিব অহংকারী, নিজবি, পৌরুষশূন্য, অন্যদিকে শাফিক উচ্ছল এবং রসিক।'}, {'question': "সবুজের বাবার আচরণ 'অপরিচিতা' গল্পের কোন চরিত্রকে স্মরণ করায়?", 'answer': "সবুজের বাবার আচরণ 'অপরিচিতা' গল্পের অনুপমের মামাকে স্মরণ করায়।"}]}, page_content='ামার সঙ্গে মা একযোগে বিস্তর হাসিলেন\n(ঘ) ঠাট্টার সম্পর্ককে স্থায়ী করিবার ইচ্ছা আমার নাই উত্তর: ক\n\n26 [নেন\nঅনলাইন ব্যাট" 10\n19475\n\nনিচের উদ্দীপকটি পড়ে ৩৬ ও ৩৭ নম্বর প্রশ্নের উত্তর দাও:\n\nশাওনের বিয়ে চুড়ান্ত হয় অন্যার সাথে। যৌতুকে

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: অনুপমের মামা ধনী ঘরের মেয়ে পছন্দ করতেন। (উত্তর: (ক) ধনী)


In [136]:
query = "বিয়ের প্রস্তাব ব্যর্থ হওয়ার পর অনুপম কনেটিকে কীভাবে বর্ণনা করে?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '25a205bc-4ab6-4489-bd68-b4b24642df0c', 'source': 'text', 'title': "Anupam's Relationship with His Uncle", 'questionanswer': [{'question': "Who is Anupam's guardian?", 'answer': 'His uncle.'}, {'question': 'অনুপমের মামা কত বছর বড়?', 'answer': 'ছয় বছর।'}, {'question': "What is a characteristic of Anupam's uncle?", 'answer': 'তিনি অনুপমের সংসার আঁকড়ে আছেন।'}, {'question': "What type of son-in-law does Anupam's uncle prefer?", 'answer': 'যার তেজ নেই এবং টাকা দিতে কসুর করবে না।'}]}, page_content='মামাকে ভুলতে দেন না\n\n॥. অনুপমকে ভুলতে দেন না\n\nনিচের কোনটি সঠিক?\n\n(ক), (খ)।, | (গ)।, | (ঘ)।,11, 1\n\n38 [15026]\nঅনলাইন ব্যাট" 10\n10401০:\n\n৯২। কোন তথ্যগুলো অনুপমের মামার ক্ষেত্রে প্রযোজ্য ঃ\n\n1. মামাই অনুপমের অভিভাবক\n\n1॥. তিনি অনুপমের চেয়ে বড়জোর বছর ছয়েকের বড়\n\n1. ফন্ধুর বালির মতো তিনি অনুপমের সংসার আঁকড়ে আছেন\n\nনিচের কোনটি সঠিক?\n\n(ক), (খ)।, | (গ)।, | (ঘ)।,11, 1\n৯৩। মামার পছন্দের বেয়াই এমন-\n\n1. যার তেজ নেই\n\n1. টাকা দিতে কসুর করবে না\

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: বিয়ের প্রস্তাব ব্যর্থ হওয়ার পর অনুপম কনেটিকে কল্পনায় ধরে রাখে এবং তার প্রতি গভীর অনুভূতি ব্যক্ত করে। সে মনে করে যে কনেটি তাকে এখনও মনে রেখেছে এবং দিনের পর দিন তার দুঃখের কথা চিন্তা করে। অনুপমের মনে কনেটির ছবি আছে, যা সে কখনো চোখে দেখেনি, ফলে তার অনুভূতি সম্পূর্ণরূপে কল্পনাপ্রসূত। গল্পে বর্ণিত হয়, সে কনেটির জন্য ডাক শুনতে এবং তার মনে থাকা আশ্চর্য রূপের কথা ভেবে উতলা হয়ে থাকে। এই সম্পর্কের কারণে অনুপমের মানসিক ভঙ্গুরতা এবং দুর্বলতা প্রকাশ পায়, কারণ বিয়ের সময় অন্যান্যদের নিষ্ঠুরতা এবং যৌতুকের কারণে তার বিয়ে ভেঙে যায়। এই ঘটনা অনুপমকে ভেঙে দেয় এবং তার জীবনকে নতুন দিকে প্রবাহিত করতে বাধা সৃষ্টি করে।


In [137]:
query = "What role did Harish play in the story?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '519dc0ed-5c7d-4ca2-83db-82e46200d48c'}, page_content='করিয়া পরিয়াছিল তাহাও ঠিক করিয়া বলিতে পারিব না। এটা খুব সত্য যে, তার বেশে ভূষায় এমন কিছুই ছিল\n\n14 [নেন\nঅনলাইন ব্যাচ" 10\n10401০:\n\nনা যেটা তাহাকে ছাড়াইয়া বিশেষ করিয়া চোখে পড়িতে পারে। সে নিজের চারি দিকের সকলের চেয়ে অধিক -\nরজনীগন্ধার শুভ্র মঞ্জরীর মতো সরল বৃন্তটির উপরে দীড়াইয়া, যে গাছে ফুটিয়াছে সে গাছকে সে একেবারে\nঅতিক্রম করিয়া উঠিয়াছে। সঙ্গে দুটি-তিনটি ছোটো ছোটো মেয়ে ছিল, তাহাদিগকে লইয়া তাহার হাসি এবং\nকথার আর অন্ত ছিল না। আমি হাতে একখানা বই লইয়া সে দিকে কান পাতিয়া রাখিয়াছিলাম। যেটুকু কানে\nআসিতেছিল সে তো সমস্তই ছেলেমানুষদের সঙ্গে ছেলেমানুষি কথা। তাহার বিশেষত্ব এই যে, তাহার মধ্যে\nবয়সের তফাত কিছুমাত্র ছিল না- ছোটোদের সঙ্গে সে অনায়াসে এবং আনন্দে ছোটো হইয়া গিয়াছিল। সঙ্গে\nকতকগলি ছবিওয়ালা ছেলেদের গল্পের বই-তাহারই কোন একটা বিশেষ গল্প শোনাইবার জন্য মেয়েরা তাহাকে\nধরিয়া পড়িল। এ গল্প নিশ্চয় তারা বিশ-পঁচিশ বার শনিয়াছে। মেয়েদের কেন যে এত আগ্রহ তাহা বুঝিলাম।\nসেই সুধাকঠের 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: In the story, Harish plays the role of a friend and supporter of Anupam. He is depicted as a lively character who helps in introducing the idea of marriage to Anupam by speaking highly of a suitable girl he knows, which catalyzes the events leading to Anupam's engagement. Harish's engaging and humorous nature creates a conducive atmosphere for Anupam and influences the family's discussions about marriage. Ultimately, he serves as a bridge that leads Anupam towards considering the possibility of love and companionship, but also highlights the social constraints that Anupam faces.


In [157]:
query = "Who said “খাঁটি সোনা বটে” and about whom?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: “খাঁটি সোনা বটে” এই কথাটি অনুপমের সম্পর্কে বলা হয়েছিল।


In [139]:
query = "গল্পে “ফল্গু নদী” উপমাটি কী বোঝাতে ব্যবহার করা হয়েছে?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '2d8550db-5846-42cb-995a-71afb32c2624', 'source': 'text', 'title': "Anupam's Self-Critique on Life", 'questionanswer': [{'question': 'অনুপমের বয়স কত বছর?', 'answer': 'পঁচিশ, ছাব্বিশ, সাতাশ, আটাশ মূল শব্দ।'}, {'question': 'অনুপমের জীবন সম্পর্কে কী বলা হয়েছে?', 'answer': 'তার জীবনটি নিতান্তই তুচ্ছ, পরিমাণ ও গুণ উভয় দিক দিয়েই।'}, {'question': 'গল্পের কথক চরিত্রের নাম কী?', 'answer': 'অনুপম।'}, {'question': 'অনুপমের ব্যবহৃত উপমা কী বোঝায়?', 'answer': 'নিজের নিম্ষল জীবনকে বোঝাতে গুটির উপমা ব্যবহার করেছেন।'}]}, page_content='ক?\n\nক।। ও ॥ খ। ॥ও || গ।। ও | ঘ।1,1 ও |\n\n&. অনুপমের বয়স কত বছর?\nক) পঁচিশ খ) ছাব্বিশ গ) সাতাশ ঘ) আটাশ মূল শব্দ\n\nএ জীবনটা না দৈর্ঘ্যের হিসাবে বড়ো, না\nগুণের হিসাবে\n\nশব্দার্থ ও টীকা\n\nশব্দের অর্থ ও ব্যাখ্যা\n\nগল্পের কথক চরিত্র অনুপমের আত্মসমালোচনা। পরিমাণ ও\nগুণ উভয় দিক দিয়েই যে তার জীবনটি নিতান্তই তুচ্ছ সে\nকথাই এখানে ব্যক্ত হয়েছে।\n\nগুটি এক সময় পূর্ণ ফলে পরিণত হয়। কিন্তু গুটিই যদি\nফলের মতো হয় তাহলে তার অসম্পূর্ণ

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: গল্পে “ফল্গু নদী” উপমাটি মূলত একটি অন্তর্নিহিত দৃষ্টান্ত হিসেবে ব্যবহৃত হয়েছে। এটি বোঝাতে চেয়েছে যে, নদীটি অন্যান্য নদীর মতো প্রবাহিত হয় না, বরং কিছু অংশে শুকিয়ে যায়। এই উপমাটি দ্বারা অনুপমের জীবনকে নির্দেশ করা হয়েছে, যা তার অসম্পূর্ণতা এবং নিম্মলসতা প্রতীকেরূপে গৃহীত হয়েছে। ফলস্বরূপ, এটি জীবন ও তার গতিপ্রবাহের অব্যাহত চিত্র তুলে ধরে, যেখানে কিছু অংশ অদৃশ্য বা নিষ্ক্রিয় হয়ে পড়ে, ঠিক যেমন ফল্গু নদী।


In [140]:
query = "গল্পে গয়না পরীক্ষা করার সময় কী ঘটেছিল?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': 'b0178efd-5158-4d8b-a020-3b759ebf8e88'}, page_content='ন\nমানসিকতাসম্পন্ন মানুষের ঘরে মেয়ে দেবেন না। কন্যাপক্ষের সমস্ত গহনা একে একে পরীক্ষা করা শেষ হলে\n\n46 [15026]\nঅনলাইন ব্যাট" 10\n10401০:\n\nশস্তুনাথ সেন একজোড়া কানের দুল সেকরাকে পরীক্ষা করতে বলেন। সেকরা জানায় এ দুলে সোনার পরিমাণ\nঅনেক কম আছে। এঁ কানের দুল অনুপমের মামা মেয়েকে আশীর্বাদ করার সময় দিয়েছিলেন। শস্তুনাথ সেন\nঅনুপমের মামার হাতে কানের দুল জোড়া দিয়ে প্রশ্নোক্ত কথাটি বলেন। এ ঘটনায় অনুপমের মামা অপমানিত\nবোধ করেন।\n\nগ. উদ্দীপকের পরেশ \'অপরিচিতা\' গল্পের অনুপম চরিত্রের বিপরীত।\n\nযথার্থ মূল্যবোধসম্পন্ন মানুষ কখনই অসংগতিকে মেনে নিতে পারেন না। যদি বলিম্ ব্যক্তিত্বের অধিকারী কেউ\nহন তবে তিনি নিজেকে প্রতিষ্ঠিত করতে পারেন। আর যে ব্যক্তি বলিষ্ঠ ব্যক্তিত্বের অধিকারী হতে পারে না তার\nজীবন হয় ব্যর্থ ও হতাশাগ্রস্ত\n\nউদ্দীপকের পরেশ একজন বেসরকারি চাকরিজীবী। বাবা-মায়ের পছন্দের শিক্ষিতা সুন্দরী এক মেয়ের সঙ্গে\nবিয়ে ঠিক হয় যৌতুক নেওয়ার বিনিময়ে। সবকিছু জানার পর পরেশ বিনিময় ছাড়া বিয়েতে রাজ

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: গল্পে গয়না পরীক্ষা করার সময় শস্তুনাথ সেনের কন্যার বিয়ের অনুষ্ঠানে অনুপমের মামা একজোড়া কানের দুল পরীক্ষা করতে আসেন। সেকরা জানায় যে, কানের দুলে সোনার পরিমাণ অনেক কম আছে। এই ঘটনা শাস্তুনাথ সেনের কাছে অপমানজনক হয় এবং তিনি মেয়েকে গয়না নিয়ে অসম্মতি জানান। এই ঘটনায় অনুপমের মামা অপমানিত বোধ করেন।


In [141]:
query = "“কষ্টিপাথর” শব্দটি কোন প্রসঙ্গে ব্যবহার হয়েছে?"
result = chain.invoke(query)
print("Answer:", result)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Docs: [Document(metadata={'doc_id': '91e79153-f7fb-4d47-aaa0-b242d4fe653d'}, page_content='ন\nকরিতে পারি নাই, ক্ষমা করিবেন। রাত হইয়া গেছে, আর আপনাদের কষ্ট বাড়াইতে ইচ্ছা করি না। এখন তবে-”\n\n10 [নেন\nঅনলাইন ব্যাচ" 10\n10401০:\n\nমামা বলিলেন, “তা, সভায় চলুন, আমরা তো প্রস্তুত আছি।”\n\nশস্তুনাথ বলিলেন, “তবে আপনাদের গাড়ি বলিয়া দিই?”\n\nশস্তুনাথ কহিলেন, “ঠাট্টা তো আপনিই করিয়া সারিয়াছেন। ঠাট্টার সম্পর্কটাকে স্থায়ী করিবার ইচ্ছা আমার\nনাই।”\nমামা দুই চোখ এত বড়ো করিয়া মেলিয়া অবাক হইয়া রইলেন।\n\nশস্তুনাথ কহিলেন, “আমার কন্যার গহনা আমি চুরি করিব এ কথা যারা মনে করে তাদের হাতে আমি কন্যা\nদিতে পারি না।\n\nআমাকে একটি কথা বলাও তিনি আবশ্যক বোধ করিলেন না। কারণ, প্রমাণ হইয়া গেছে, আমি কেহই নই।\nতারপরে যা হইল সে আমি বলিতে ইচ্ছা করি না। ঝাড়লগ্ঠন ভাঙিয়া-চুরিয়া, জিনিসপত্র লণ্ডভণ্ড করিয়া,\nবরযাত্রের দল দক্ষঘজ্ঞের পালা সারিয়া বাইর\nহইয়া গেল। বাড়ি ফিরিবার সময় ব্যান্ড রসনচৌকি\nও কন্নর্ট একসঙ্গে বাজিল না এবং অভ্রের\nঝাড়গুলো আকাশের তারার উপর আপনাদের\nকর্তব্যের বরাত দিয়া কোথায় যে মহানির্বাণ লাভ

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: “কষ্টিপাথর” শব্দটি ব্যবহৃত হয়েছে স্বর্ণের অলঙ্কারের খাঁটিত্ব পরীক্ষা করার প্রসঙ্গে। এখানে বলা হয়েছে, দেওয়া-থোওয়া যে পাথরে ঘষে সোনার খাঁটিত্ব পরীক্ষা করা হয়, সেটি হচ্ছে কষ্টিপাথর।


In [143]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher

def evaluate_groundedness(answer, docs, threshold=0.7):
    """
    Check if answer is grounded in retrieved documents.
    Returns a groundedness score (0-1) and binary judgment.
    """
    retrieved_text = "\n".join([doc.page_content for doc in docs if isinstance(doc, Document)])

    # Cosine similarity
    vectorizer = TfidfVectorizer().fit([answer, retrieved_text])
    vectors = vectorizer.transform([answer, retrieved_text])
    sim_score = cosine_similarity(vectors[0], vectors[1])[0][0]

    # Fuzzy match using SequenceMatcher
    seq_score = SequenceMatcher(None, answer, retrieved_text).ratio()

    # Combine scores (you could weight them differently)
    groundedness_score = (sim_score + seq_score) / 2
    is_grounded = groundedness_score >= threshold
    return groundedness_score, is_grounded


In [144]:
def evaluate_relevance(query, docs, expected_keywords=None):
    """
    Compute relevance based on presence of expected keywords.
    If expected_keywords is None, extract top nouns from query.
    """
    from nltk import word_tokenize, pos_tag
    import re

    if expected_keywords is None:
        tokens = word_tokenize(query)
        expected_keywords = [word for word, tag in pos_tag(tokens) if tag in ("NN", "NNP")]

    retrieved_text = "\n".join([doc.page_content for doc in docs if isinstance(doc, Document)])
    keyword_matches = sum(1 for kw in expected_keywords if re.search(r'\b' + re.escape(kw) + r'\b', retrieved_text))

    relevance_score = keyword_matches / len(expected_keywords) if expected_keywords else 0
    return relevance_score


In [145]:
def evaluate_rag(query, answer, docs, expected_keywords=None):
    grounded_score, is_grounded = evaluate_groundedness(answer, docs)
    relevance_score = evaluate_relevance(query, docs, expected_keywords)

    return {
        "query": query,
        "answer": answer,
        "groundedness_score": round(grounded_score, 3),
        "is_grounded": is_grounded,
        "relevance_score": round(relevance_score, 3),
    }


In [155]:
import json
from base64 import b64decode

from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

from langdetect import detect
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher
import re
import nltk
from nltk import word_tokenize, pos_tag

# Make sure NLTK is downloaded
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

# ========== 1. Optimize Query ==========

def optimize_query(query: str) -> dict:
    system_prompt = """
You are an expert in multilingual query optimization for a RAG (Retrieval-Augmented Generation) pipeline that supports both English and Bengali (Bangla).

Your job is to:
1. Reformulate the original question clearly and naturally in its own language (English or Bengali) for use with a vector-based retriever.
2. Extract concise and essential keywords (nouns, names, concepts) for use with a BM25 retriever. Only include important terms, and return them as a space-separated string — do not translate the language.

Output JSON must follow this format:
{
  "vector_query": "rephrased full question (same language)",
  "bm25_keywords": "keyword1 keyword2 keyword3"
}
Only return valid JSON.
"""

    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_prompt.strip()),
        HumanMessage(content=f"Original Query: {query}")
    ])

    llm = ChatOpenAI(model="gpt-4o-mini")
    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({"query": query})
    
    try:
        result = json.loads(response)
        return {
            "vector_query": result.get("vector_query", query).strip(),
            "bm25_keywords": result.get("bm25_keywords", query).strip()
        }
    except json.JSONDecodeError:
        print("[Warning] LLM did not return valid JSON. Falling back to original query.")
        return {
            "vector_query": query,
            "bm25_keywords": query
        }

# ========== 2. Simulated Retriever Calls ==========

# You must define your own retrievers (mocked here)
mv_retriever = pd_retriever = bm25_retriever = bm25_docs_retriever = lambda q: [Document(page_content="অনুপমের বাবা ওকালতি করতেন।", metadata={"source": "অপরিণতা"})]

# ========== 3. Combine Retrieved Documents ==========

def combine_retrievers(query):
    optimized_queries = optimize_query(query)
    vector_query = optimized_queries["vector_query"]
    bm25_keywords = optimized_queries["bm25_keywords"]

    mv_docs = mv_retriever(vector_query)
    pd_docs = pd_retriever(vector_query)
    bm25_docs = bm25_retriever(bm25_keywords)
    bm25_summary_docs = bm25_docs_retriever(bm25_keywords)

    seen = set()
    combined_docs = []
    for doc in pd_docs + bm25_docs + bm25_summary_docs:
        if doc.page_content not in seen:
            seen.add(doc.page_content)
            combined_docs.append(doc)

    return combined_docs

# ========== 4. Parse Documents ==========

def parse_docs(docs):
    b64 = []
    text = []
    for doc in docs:
        if not isinstance(doc, Document):
            continue
        try:
            b64decode(doc.page_content)
            b64.append(doc.page_content)
        except Exception:
            text.append(doc)
    return {"images": b64, "texts": text}

# ========== 5. Detect Language ==========

def detect_lang(text):
    try:
        return detect(text)
    except:
        return "en"

# ========== 6. Build Prompt ==========

def build_prompt(kwargs):
    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]
    lang = detect_lang(user_question)

    context_text = ""
    for text_element in docs_by_type["texts"]:
        source = text_element.metadata.get("source", "unknown")
        context_text += f"[Source: {source}]\n{text_element.page_content}\n\n"

    prompt_template = f"""
You are an AI assistant answering questions in Bengali or English based on context from textbooks or official documents.

- Use only the most relevant parts of the context to answer.
- Maintain structure and accuracy when referring to tables, names, or numbers.
- Ignore irrelevant or duplicate context.
- Respond in the same language as the user question ("{lang}").

Context:
{context_text}

Question:
{user_question}
"""

    prompt_content = [{"type": "text", "text": prompt_template}]
    return ChatPromptTemplate.from_messages([HumanMessage(content=prompt_content)])

# ========== 7. LLM Chain ==========

llm = ChatOpenAI(model="gpt-4o-mini")

chain = (
    {
        "context": RunnableLambda(combine_retrievers) | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | llm
    | StrOutputParser()
)

# ========== 8. RAG Evaluation ==========

def evaluate_groundedness(answer, docs, threshold=0.7):
    retrieved_text = "\n".join([doc.page_content for doc in docs if isinstance(doc, Document)])
    vectorizer = TfidfVectorizer().fit([answer, retrieved_text])
    vectors = vectorizer.transform([answer, retrieved_text])
    sim_score = cosine_similarity(vectors[0], vectors[1])[0][0]
    seq_score = SequenceMatcher(None, answer, retrieved_text).ratio()
    groundedness_score = (sim_score + seq_score) / 2
    is_grounded = groundedness_score >= threshold
    return groundedness_score, is_grounded

def evaluate_relevance(query, docs, expected_keywords=None):
    if expected_keywords is None:
        tokens = word_tokenize(query)
        expected_keywords = [word for word, tag in pos_tag(tokens) if tag in ("NN", "NNP")]
    retrieved_text = "\n".join([doc.page_content for doc in docs if isinstance(doc, Document)])
    keyword_matches = sum(1 for kw in expected_keywords if re.search(r'\b' + re.escape(kw) + r'\b', retrieved_text))
    relevance_score = keyword_matches / len(expected_keywords) if expected_keywords else 0
    return relevance_score

def evaluate_rag(query, answer, docs, expected_keywords=None):
    grounded_score, is_grounded = evaluate_groundedness(answer, docs)
    relevance_score = evaluate_relevance(query, docs, expected_keywords)

    return {
        "query": str(query),
        "answer": str(answer),
        "groundedness_score": float(round(grounded_score, 3)),
        "is_grounded": bool(is_grounded),
        "relevance_score": float(round(relevance_score, 3)),
    }

# ========== 9. Run a Sample Query ==========

query = "অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?"
result = chain.invoke(query)
print("Answer:", result)

# Get retrieved docs and evaluate
docs_used = combine_retrievers(query)
evaluation = evaluate_rag(query, result, docs_used, expected_keywords=["অনুপম", "বাবা", "ওকালতি"])
print("\nRAG Evaluation:")
print(json.dumps(evaluation, indent=2, ensure_ascii=False))


[nltk_data] Downloading package punkt to /home/rudra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/rudra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: অনুপমের বাবা ওকালতি করতেন।


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



RAG Evaluation:
{
  "query": "অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?",
  "answer": "অনুপমের বাবা ওকালতি করতেন।",
  "groundedness_score": 1.0,
  "is_grounded": true,
  "relevance_score": 0.333
}


In [156]:
test_cases = [
    {
        "query": "অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?",
        "expected_keywords": ["অনুপম", "বাবা", "ওকালতি"]
    },
    {
        "query": "গল্পে হরিমতীর চরিত্র কেমন ছিল?",
        "expected_keywords": ["হরিমতী", "চরিত্র"]
    },
    {
        "query": "ফল্গু নদী উপমার তাৎপর্য কী?",
        "expected_keywords": ["ফল্গু", "নদী", "উপমা", "তাৎপর্য"]
    },
    {
        "query": "দীপুর চাচা কেন বিয়েতে রাজি ছিলেন না?",
        "expected_keywords": ["দীপু", "চাচা", "বিয়ে"]
    },
    {
        "query": "“কষ্টিপাথর” শব্দটি কোন প্রসঙ্গে ব্যবহার হয়েছে?",
        "expected_keywords": ["কষ্টিপাথর", "প্রসঙ্গ"]
    },
    {
        "query": "গল্পে গয়না পরীক্ষা করার সময় কী ঘটেছিল?",
        "expected_keywords": ["গয়না", "পরীক্ষা", "ঘটনা"]
    },
    {
        "query": "গল্পে “ফল্গু নদী” উপমাটি কী বোঝাতে ব্যবহার করা হয়েছে?",
        "expected_keywords": ["ফল্গু", "নদী", "উপমা", "অর্থ"]
    },
    {
        "query": "Who said “খাঁটি সোনা বটে” and about whom?",
        "expected_keywords": ["খাঁটি", "সোনা", "quote", "who", "said"]
    },
    {
        "query": "What role did Harish play in the story?",
        "expected_keywords": ["Harish", "role", "story"]
    },
    {
        "query": "বিয়ের প্রস্তাব ব্যর্থ হওয়ার পর অনুপম কনেটিকে কীভাবে বর্ণনা করে?",
        "expected_keywords": ["বিয়ে", "প্রস্তাব", "অনুপম", "কনে", "বর্ণনা"]
    },
    {
        "query": "অনুপমের মামা কেমন জামাই পছন্দ করতেন?",
        "expected_keywords": ["অনুপম", "মামা", "জামাই", "পছন্দ"]
    },
    {
        "query": "What do we learn about Anupam’s uncle’s personality during the bride-seeing visit?",
        "expected_keywords": ["Anupam", "uncle", "personality", "bride"]
    },
    {
        "query": "Who was Anupam’s guardian and how did he influence Anupam’s life decisions?",
        "expected_keywords": ["Anupam", "guardian", "life", "decisions"]
    },
    {
        "query": "হরিশ কনের বাবার সম্পর্কে কী কী বলেছেন?",
        "expected_keywords": ["হরিশ", "কনের", "বাবা", "মতামত"]
    }
]

results = []
for test in test_cases:
    query = test["query"]
    expected_keywords = test.get("expected_keywords", [])

    # Get answer from RAG
    answer = chain.invoke(query)

    # Combine documents used in generation
    docs_used = combine_retrievers(query)

    # Evaluate RAG output
    evaluation = evaluate_rag(query, answer, docs_used, expected_keywords=expected_keywords)
    results.append(evaluation)

    # Print individual result
    print(f"\nQuery: {query}")
    print(f"Answer: {answer}")
    print(json.dumps(evaluation, indent=2, ensure_ascii=False))
avg_grounded = sum(r["groundedness_score"] for r in results) / len(results)
avg_relevance = sum(r["relevance_score"] for r in results) / len(results)

print("\n--- Evaluation Summary ---")
print(f"Average Groundedness Score: {avg_grounded:.3f}")
print(f"Average Relevance Score:    {avg_relevance:.3f}")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query: অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?
Answer: অনুপমের বাবা ওকালতি করতেন।
{
  "query": "অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?",
  "answer": "অনুপমের বাবা ওকালতি করতেন।",
  "groundedness_score": 1.0,
  "is_grounded": true,
  "relevance_score": 0.333
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query: গল্পে হরিমতীর চরিত্র কেমন ছিল?
Answer: গল্পে হরিমতীর চরিত্র সম্পর্কে নির্দিষ্ট কোনো তথ্য নেই। তবে, সাধারণত হরিমতীর চরিত্রটি গঠনমূলক ও প্রভাবশালী হতে পারে, যা গল্পের মূল পটভূমি এবং চরিত্রগুলোর সঙ্গে সম্পর্কিত। অনুগ্রহ করে যদি আরো বিস্তারিত তথ্য থাকে তবে তা উল্লেখ করুন।
{
  "query": "গল্পে হরিমতীর চরিত্র কেমন ছিল?",
  "answer": "গল্পে হরিমতীর চরিত্র সম্পর্কে নির্দিষ্ট কোনো তথ্য নেই। তবে, সাধারণত হরিমতীর চরিত্রটি গঠনমূলক ও প্রভাবশালী হতে পারে, যা গল্পের মূল পটভূমি এবং চরিত্রগুলোর সঙ্গে সম্পর্কিত। অনুগ্রহ করে যদি আরো বিস্তারিত তথ্য থাকে তবে তা উল্লেখ করুন।",
  "groundedness_score": 0.067,
  "is_grounded": false,
  "relevance_score": 0.0
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...


Query: What do we learn about Anupam’s uncle’s personality during the bride-seeing visit?
Answer: The provided context does not include any information about Anupam’s uncle or his personality during the bride-seeing visit. Therefore, I cannot provide an answer to your question based on the available information.
{
  "query": "What do we learn about Anupam’s uncle’s personality during the bride-seeing visit?",
  "answer": "The provided context does not include any information about Anupam’s uncle or his personality during the bride-seeing visit. Therefore, I cannot provide an answer to your question based on the available information.",
  "groundedness_score": 0.012,
  "is_grounded": false,
  "relevance_score": 0.0
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query: Who was Anupam’s guardian and how did he influence Anupam’s life decisions?
Answer: Anupam's guardian was his father, who practiced as a lawyer. His profession likely influenced Anupam's life decisions regarding his education and career aspirations.
{
  "query": "Who was Anupam’s guardian and how did he influence Anupam’s life decisions?",
  "answer": "Anupam's guardian was his father, who practiced as a lawyer. His profession likely influenced Anupam's life decisions regarding his education and career aspirations.",
  "groundedness_score": 0.016,
  "is_grounded": false,
  "relevance_score": 0.0
}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Query: হরিশ কনের বাবার সম্পর্কে কী কী বলেছেন?
Answer: প্রদানকৃত প্রসঙ্গে হরিশ কনের বাবার সম্পর্কে কোনো তথ্য নেই। তাই এ প্রশ্নের সঠিক উত্তর দিতে পারছি না।
{
  "query": "হরিশ কনের বাবার সম্পর্কে কী কী বলেছেন?",
  "answer": "প্রদানকৃত প্রসঙ্গে হরিশ কনের বাবার সম্পর্কে কোনো তথ্য নেই। তাই এ প্রশ্নের সঠিক উত্তর দিতে পারছি না।",
  "groundedness_score": 0.136,
  "is_grounded": false,
  "relevance_score": 0.0
}

--- Evaluation Summary ---
Average Groundedness Score: 0.211
Average Relevance Score:    0.056
